# Wrangle OpenStreetMap data
[Cédric Campguilhem](https://github.com/ccampguilhem/Udacity-DataAnalyst), August 2017

<a id="Top"/>

## Table of contents
- [Introduction](#Introduction)
- [Project organisation](#Organisation)
- [Map area selection](#Area selection)
- [XML data structure](#XML data structure)
- [Data quality audit](#Data quality)
    - [Validity](#Data validity)
    - [Accuracy](#Data accuracy)
    - [Completeness](#Data completeness)
    - [Consistency](#Data consistency)
    - [Uniformity](#Data uniformity)
    - [Conclusion](#Audit conclusion)
- [Data cleaning](#Data cleaning)
    - [Method](#Method)
    - [Converting to dictionnary-like structure](#Converting to dictionnary-like structure)
    - [Cleaning accuracy issues](#Cleaning accuracy issues)
    - [Cleaning completeness issues](#Cleaning completeness issues)
    - [Cleaning consistency issues](#Cleaning consistency issues)
    - [Cleaning uniformities issues](#Cleaning uniformities issues)
    - [Conclusion](#Cleaning conclusion)
- [Data export](#Data export)
    - [To JSON and MongoDB](#JSON MongoDB)
    - [To csv and SQLite](#csv SQLite)
- [Appendix](#Appendix)

<a id="Introduction"/>

## Introduction *[top](#Top)*

This project is related to Data Wrangling with MongoDB course for Udacity Data Analyst Nanodegree program.
The purpose of this project is to:

- Collect data from [OpenStreetMap](https://www.openstreetmap.org) web services.
- Clean the data by fixing few issues introduced by users.
- Store the dataset in a database to make any further analysis easier.

OpenStreetMap is open data, licensed under the Open Data Commons Open Database License (ODbL) by the OpenStreetMap Foundation (OSMF). 

This project cover various aspects of data wrangling phase:
- **screen scraping** with [Requests](http://requests.readthedocs.io/en/master/), an http Python library for making requests on web services,
- **parsing** XML files with iterative and SAX parsers with Python standard library [xml.etree.ElementTree](https://docs.python.org/2/library/xml.etree.elementtree.html?highlight=iterparse#module-xml.etree.ElementTree) and [xml.sax](https://docs.python.org/2/library/xml.sax.html),
- **auditing** (validity, accuracy, completeness, consistency and uniformity) and **cleaning** data with Python,
    - validity: does data conform to a schema ?
    - accuracy: does data conform to gold standard (a dataset we trust) ?
    - completeness: do we have all records ?
    - consistency: is dataset providing contradictory information ?
    - uniformity: are all data provided in the same units ?
- **storing** data into SQL database (SQLite) with Python [sqlite3](https://docs.python.org/2/library/sqlite3.html) module and [MongoDG](https://www.mongodb.com/) no-SQL database.
- exploring dataset **statistics** as per project requirements (size of the file, number of unique users, number of nodes and ways, number of chosen type of nodes, like cafes, shops etc.)

The storing step will make use of [csv](https://docs.python.org/2/library/csv.html?highlight=csv#module-csv) and [json](https://docs.python.org/2/library/json.html?highlight=json#module-json) formats respectively for SQL and MongoDB exports.

I am already a bit familiar with SQL but I will also provide SQL output in addition to MongoDB output for the cleaned dataset.

<a id='Project organisation'/>

## Project organisation *[top](#Top)*

The project is decomposed in the following manner:

- This notebook (data_wrangling.ipynb) contains top-level code as well as results and report.
- The [data_wrangling.py](./data_wrangling.py) file is a Python export of this notebook.
- The [data_wrangling.html](./data_wrangling.html) file is a html export of this notebook.
- The [environment.yml](./environment.yml) file contains the anaconda environment I used for this project.
- The [handler.py](./handler.py) module contains a class used as content handler for SAX XML parser.
- The [utils.py](./utils.py) module contains functions used by audit classes.
- The [validity_audit.py](./validity_audit.py) module contains a callback class for validity audit.
- The [accuracy_audit.py](./accuracy_audit.py) module contains a callback class for accuracy audit.
- The [completeness_audit.py](./completeness_audit.py) module contains a callback class for completeness audit.
- The [consistency_audit.py](./consistency_audit.py) module contains a callback class for consistency audit.
- The [uniformity_audit.py](./uniformity_audit.py) module contains a callback class for uniformity audit.
- The [dictionnary_export.py](./dictionnary_export.py) module contains a callback class to export data to a dicitonnary.

In [1]:
#Enable auto-reload of modules, will help as we have a lot of modules
%load_ext autoreload
%autoreload 2

<a id="Area selection"/>

## Map area selection *[top](#Top)*

If you don't want to have details on how the data from OpenStreetMap is retrieved, you can skip this section. At the end of the processing, you should have a *data.osm* file in the same directory than this notebook.

I have made the map area selection dynamic. By configuring few variables, a different map area may be extracted from OpenStreetMap. Some pre-selections are available:

| Pre-selection | Description               | Usage               | File size (bytes) | OpenStreetMap link |
|:------------- |:------------------------- |:------------------- | -----------------:|:------------------ |
| Tournefeuille | The city I live in        | Project review      | 103 143 437       | [link](https://www.openstreetmap.org/relation/35735)
| City center   | Tournefeuille city center | Testing, debugging  | 583 419           | [link](https://www.openstreetmap.org/export#map=14/43.5848/1.3516)
| Toulouse      | Toulouse and surroundings | Benchmark           | 1 271 859 210     | [link](https://www.openstreetmap.org/search?query=toulouse#map=11/43.6047/1.4442)

The box variables are in the following order (south-west to north-east):

- minimum latitude
- minimum longitude
- maximum latitude
- maximum longitude

**Note: ** The data cleaning provided in this project works for French area, if you select a non-french area no data cleaning will be performed.

In [2]:
SELECTION = "PRESELECTED" #Update the PRESELECTION variable
#SELECTION = "USER" #Update the USER_SELECTION with the box you want
#SELECTION = "CACHE" #Use any data file present in directory
USER_SELECTION = (43.5799, 1.3434, 43.5838, 1.3496)
PRESELECTIONS = {"Tournefeuille": (43.5475, 1.2767, 43.6019, 1.3909),
                 "City center": (43.5799, 1.3434, 43.5838, 1.3496),
                 "Toulouse": (43.3871, 0.9874, 43.8221, 1.9006)}
PRESELECTION = "Tournefeuille"
TEMPLATE = \
"""
(
   node({},{},{},{});
   <;
);
out meta;
"""

I have used screen scrapping techniques presented throught the course to extract data from OpenStreetMap:

- I use the Overpass API (http://wiki.openstreetmap.org/wiki/Overpass_API)
- The query form (http://overpass-api.de/query_form.html) sends a POST request to http://overpass-api.de/api/interpreter
- From the api/interpreter we can just make a GET request which takes a data parameter containing the box selection:

```
(
   node(51.249,7.148,51.251,7.152);
   <;
);
out meta;
```

The idea is to send a http GET request using [Requests](http://requests.readthedocs.io/en/master/) and collect results in a stream. This is because the data we get from the request may be huge and may not fit into memory.

The following method `download_map_area` enables to download map area data and store it in a *data.osm* file:

In [3]:
import os
import requests


def download_map_area():
    """
    Download the map area in a file named data.osm.
    
    This function takes into account the following global variables: SELECTION, USER_SELECTION, PRESELECTIONS, 
    PRESELECTION and TEMPLATE
    
    If a http request is made, the response status code is returned, otherwise None in returned.
    If SELECTION is set to CACHE and no file is present an exception is raised.
    
    - raise ValueError: if SELECTION=CACHE and there is no cached file
    - raise ValueError: if SELECTION is not [PRESELECTED, USER, CACHE]
    - raise NameError if either of SELECTION, PRESELECTION, PRESELECTIONS, USER_SELECTION or TEMPLATE does not exist.
    - return: tuple:
        - status code or None
        - path to dataset
        - dataset file size (in bytes)
    """
    filename = "data.osm"
    if SELECTION == "CACHE":
        if not os.path.exists(filename):
            raise ValueError("Cannot use SELECTION=CACHE if no {} file exists.".format(filename))
        else:
            return None, filename, os.path.getsize(filename)
    elif SELECTION == "PRESELECTED":
        data = TEMPLATE.format(*PRESELECTIONS[PRESELECTION])
    elif SELECTION == "USER":
        data = TEMPLATE.format(*USER_SELECTION)
    else:
        raise ValueError("SELECTION=")
        
    #Get XML data
    r = requests.get('http://overpass-api.de/api/interpreter', params={"data": data}, stream=True)
    with open(filename, 'wb') as fobj:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk:
                fobj.write(chunk)
    return r.status_code, filename, os.path.getsize(filename)

In [4]:
#Download dataset
%time status_code, dataset_path, dataset_size = download_map_area()
if status_code is None:
    print "The file {} is re-used from a previous download. Its size is {} bytes.".format(dataset_path, dataset_size)
elif status_code == 200:
    print "The file {} has been successfully downloaded. Its size is {} bytes.".format(dataset_path, dataset_size)
else:
    print "An error occured while downloading the file. Http status code is {}.".format(status_code)

CPU times: user 348 ms, sys: 144 ms, total: 492 ms
Wall time: 7.23 s
The file data.osm has been successfully downloaded. Its size is 103241495 bytes.


<a id="XML data structure"/>

## XML data structure *[top](#Top)*

In the previous section, we have downloaded a dataset from OpenStreetMap web service. The XML file retrieved this way is stored in the file named *data.osm*.

In this section we are going to familiarize with the dataset to understand how it's built. As dataset may be a very large file (depending on the map area extracted) we are going to use an iterative parser that does not need to load the entire document in memory.

In [5]:
#Import the XML library
import xml.etree.cElementTree as et

from collections import Counter, defaultdict
from pprint import pprint

from IPython.core.display import display, HTML

In [6]:
#Iterative parsing
element_tags = Counter()
for (event, elem) in et.iterparse(dataset_path):
    element_tags[elem.tag] += 1
pprint(dict(element_tags))

{'member': 17264,
 'meta': 1,
 'nd': 602009,
 'node': 428787,
 'note': 1,
 'osm': 1,
 'relation': 469,
 'tag': 218929,
 'way': 71960}


In OpenStreetMap, data is structured this way:
- A **node** is a location in space defined by its latitude and longitude. It might indicate a standalone point and/or can be used to define shape of a way.
- A **way** can be either a polyline to represent roads, rivers... or a closed polygon to delimit areas (buildings, parks...).
- A **nd** is used within way to reference nodes.
- A **relation** can be defined from **member** nodes and ways to represent routes, bigger area such as regions or city boundaries.
- A **member** is a subpart of a relation pointing either to a node or a way.
- A **tag** is a (key, value) information attached to nodes, ways and relations to document in more detail the item.
- **osm** is the root node in .osm files.
- **note** and **meta** are metadata.

We are now going to parse the XML file again to get the full path of each tag in the dataset. We need to use a SAX parser with a custom handler. Do not pay attention to callbacks, this will be explained in next sections.

In [7]:
import xml.sax
from handler import OpenStreetMapXmlHandler

We can now use the handler in SAX parsing:

In [8]:
parser = xml.sax.make_parser()
with OpenStreetMapXmlHandler() as handler:
    parser.setContentHandler(handler)
    parser.parse(dataset_path)

In [9]:
#Get tag counts
pprint(handler.getTagsCount())

{u'member': 17264,
 u'meta': 1,
 u'nd': 602009,
 u'node': 428787,
 u'note': 1,
 u'osm': 1,
 u'relation': 469,
 u'tag': 218929,
 u'way': 71960}


The returned tag count is the same than the one we have calculated using `et.iterparse`.

In [10]:
#Get tag ancestors
pprint(handler.getTagsAncestors())

{u'member': set([u'osm.relation']),
 u'meta': set([u'osm']),
 u'nd': set([u'osm.way']),
 u'node': set([u'osm']),
 u'note': set([u'osm']),
 u'osm': set(['']),
 u'relation': set([u'osm']),
 u'tag': set([u'osm.node', u'osm.relation', u'osm.way']),
 u'way': set([u'osm'])}


As we discussed later on:
- **osm** element has no ancestor (it's root element)
- **meta** and **note** only appear in **osm** element
- **node**, **way** and **relation** are direct children of **osm**
- **tag** can be used to document any of **node**, **way** and **relation**
- **member** are only used in **relation** elements (to reference either nodes, ways or other relations)
- **nd** are only used in **way** elements (to reference nodes)

Such result will help us a lot when auditing [data quality](#Data quality).

<a id='Data quality'/>

## Data quality audit *[top](#Top)*

This chapter is divided into 5 sections for each kind of data quality audit:
- [Validity](#Data validity)
- [Accuracy](#Data accuracy)
- [Completeness](#Data completeness)
- [Consistency](#Data consistency)
- [Uniformity](#Data uniformity)

<a id='Data validity'/>

### Validity *[audit](#Data quality)*

Validity is about compliance to a schema. The data we have retrieved from OpenStreetMap servers is a XML file. It exists techniques to validate XML structures such as XML Schema. We won't use such technique here because schema is relatively simple and because XML files can be large enough so we want to stick to using SAX parser.

Actually, the SAX content handler that has been introduced in previous [section](#XML data structure) will be helpful here as it's already able to list ancestors for each element. We can then define a schema in a similar form and compare both to see if there is any issue.

The schema is a dictionnary structured this way:
- key: element tag
- value: dictionnary with the following keys / values:
    - *ancestors*: List of any acceptable ancestor path. For example, the path ('osm.way') means that element shall be a children of a way element which itself is a children of a osm element.
    - *minOccurences*: minimum number of element in the dataset (greater or equal to 0), optional
    - *maxOccurences*: maximum number of element in the dataset (greater or equal to 1), optional
    - *requiredAttributes*: list of attribute names that shall be defined for element
    - *requiredChildren*: list of required children element
    - *attributesFuncs*: list of callable objects to be run on the element attributes for further checks

In [11]:
import functools

#Function to check numbers
check_digit = lambda name, attr: attr[name].isdigit()
check_id_digit = functools.partial(check_digit, 'id')
check_uid_digit = functools.partial(check_digit, 'uid')
check_ref_digit = functools.partial(check_digit, 'ref')

#Define a schema
schema = {
    #osm is root node. There shall be exactely one.
    'osm': { 
        'ancestors': {''}, 
        'minOccurences': 1,
        'maxOccurences': 1},
    #meta shall be within osm element. There shall be exactely one of those.
    'meta': {
        'ancestors': {'osm'},
        'minOccurences': 1,
        'maxOccurences': 1},
    #meta shall be within osm element. There shall be exactely one of those.
    'note': {
        'ancestors': {'osm'},
        'minOccurences': 1,
        'maxOccurences': 1},        
    #node shall be within osm element. A node shall have id, lat (latitude) and lon (longitude) attributes.
    #Additionally, lat shall be in the range [-90, 90] and longitude in the range [-180, 180]. Id shall be a digit 
    #number
    'node': {
        'ancestors': {'osm'},
        'requiredAttributes': ['id', 'lat', 'lon', 'uid'],
        'attributesFuncs': [lambda attr: -90 <= float(attr['lat']) <= 90, 
                            lambda attr: -180 <= float(attr['lon']) <= 180,
                            check_id_digit,
                            check_uid_digit]},
    #way shall be within osm element. A way shall have id attribute. It shall have at least one nd children.
    #id shall be a digit.
    'way': {
        'ancestors': {'osm'},
        'requiredAttributes': ['id', 'uid'],
        'requiredChildren': ['nd'],
        'attributesFuncs': [check_id_digit, check_uid_digit]},
    #nd shall be within way element. A nd shall have ref attribute. ref attribute shall be a digit.
    'nd': {
        'ancestors': {'osm.way'},
        'requiredAttributes': ['ref'],
        'attributesFuncs': [check_ref_digit]},
    #relation shall be within a osm element. It shall have a id attribute and at least one member children. id shall
    #be a digit
    'relation': {
        'ancestors': {'osm'},
        'requiredAttributes': ['id', 'uid'],
        'requiredChildren': ['member'],
        'attributesFunc': [check_id_digit, check_uid_digit]},
    #member shall be within a relation element. It shall have type, ref and role attributes. The type attribute shall
    #be either way or node. The ref attribute shall be a digit.
    'member': {
        'ancestors': {'osm.relation'},
        'requiredAttributes': ['type', 'ref', 'role'],
        'attributesFuncs': [lambda attr: attr['type'] in ['way', 'node', 'relation'],
                            check_ref_digit]},
        
    #tag shall be within node, way or relation. It shall have k and v attributes.
    'tag': {
        'ancestors': {'osm.node', 'osm.way', 'osm.relation'},
        'requiredAttributes': ['k', 'v']},
    }

In order to have this schema validated, we are going to create a callback to be passed to SAX content handler we have created earlier:

In [12]:
from validity_audit import DataValidityAudit

We create a function that will help us parsing and autiting the data:

In [13]:
import tabulate

#Define a method to parse and audit
def parse_and_audit(dataset_path, audit=None):
    """
    Parse XML dataset and perform audit quality.
    
    - dataset_path: path to the dataset to be parsed and audited
    - audit: a sequence of audit objects
    - return: sequence of nonconformities
    """
    with OpenStreetMapXmlHandler() as handler:
        if audit is not None:
            for obj in audit:
                handler.registerStartEventCallback(obj.startEventCallback)
                handler.registerEndEventCallback(obj.endEventCallback)
        parser = xml.sax.make_parser()
        parser.setContentHandler(handler)
        parser.parse(dataset_path)
    nonconformities = []
    if audit is not None:
        for obj in audit:
            nonconformities.extend(obj.getNonconformities())
    return nonconformities

In [14]:
#Parse and audit
audit = [DataValidityAudit(schema)]
%time nonconformities = parse_and_audit(dataset_path, audit)
display(HTML(tabulate.tabulate(nonconformities, tablefmt='html')))

CPU times: user 15.2 s, sys: 52 ms, total: 15.3 s
Wall time: 15.4 s


The returned list above shall be empty. It means that no nonconfirmity has been detected for validity audit. The data we get from OpenStreetMap may be trusted in terms of schema compliance.

The `%timeit` Jupyter magic command enables to monitor how much time it takes to parse and audit the data. As a reference it takes approximately 15 seconds to parse and audit the dataset of around 100 Mb.

<a id='Data accuracy'/>

### Accuracy *[audit](#Data quality)*

Accuracy is a measurement of coformity with gold standard. On a dataset such as the one from OpenStreetMap it may be difficult to find a gold standard. We are then going to limit this audit to values that are sometimes provided in the dataset for items which represents a town:
- INSEE indentifier (ref:INSEE in the above example)
- Population
- Date of last census (source:population in the above example)

Here is an example:

```xml
<node id="26691412" lat="43.5827846" lon="1.3466543" version="17" timestamp="2017-08-22T17:20:54Z" changeset="51349577" uid="6523296" user="ccampguilhem">
    <tag k="addr:postcode" v="31170"/>
    <tag k="name" v="Tournefeuille"/>
    <tag k="name:fr" v="Tournefeuille"/>
    <tag k="name:oc" v="TornafuÃ¨lha"/>
    <tag k="place" v="town"/>
    <tag k="population" v="26674"/>
    <tag k="ref:FR:SIREN" v="213105570"/>
    <tag k="ref:INSEE" v="31557"/>
    <tag k="source:population" v="INSEE 2014"/>
    <tag k="wikidata" v="Q328022"/>
    <tag k="wikipedia" v="fr:Tournefeuille"/>
</node>
```

But this information may also be attached to a relation element instead:
```xml
<relation id="158881" version="20" timestamp="2017-06-22T16:33:19Z" changeset="49751028" uid="94578" user="andygol">
    <member type="node" ref="534672451" role="admin_centre"/>
    <member type="way" ref="36353842" role="outer"/>
    <member type="way" ref="166581580" role="outer"/>
    ...
    <member type="way" ref="502733025" role="outer"/>
    <member type="way" ref="502733024" role="outer"/>
    <member type="way" ref="36353843" role="outer"/>
    <tag k="addr:postcode" v="31820"/>
    <tag k="admin_level" v="8"/>
    <tag k="boundary" v="administrative"/>
    <tag k="name" v="Pibrac"/>
    <tag k="name:fr" v="Pibrac"/>
    <tag k="name:ru" v="Пибрак"/>
    <tag k="name:uk" v="Пібрак"/>
    <tag k="name:zh" v="皮布拉克"/>
    <tag k="population" v="8091"/>
    <tag k="ref:FR:SIREN" v="213104177"/>
    <tag k="ref:INSEE" v="31417"/>
    <tag k="source:population" v="INSEE 2013"/>
```

Our audit code shall take this into account.

For this example, I have updated the OpenStreetMap database manually to match official data published by [INSEE](https://www.insee.fr/en/accueil). I will use INSEE data as gold standard (see [here](https://www.insee.fr/fr/statistiques/1405599?geo=COM-31557+COM-31291+COM-31149+COM-31424+COM-31157+COM-31417)). The last census in my region is from 2014.

We are going to define a gold standard in a dictionnary for few towns in the surrounding of Tournefeuille. If you have selected a user-defined area map, it may not be suitable to you:

In [15]:
#Used to convert digit in XML with thoudand separators into a Python integer
convert_to_int = lambda x: int(x.replace(" ", ""))

gold_standard_insee = {
    u'Tournefeuille': {
        'population': (convert_to_int, 26674),
        'source:population': (str, 'INSEE 2014'),
        'ref:INSEE': (convert_to_int, 31557)},
    u'Léguevin': {
        'population': (convert_to_int, 8892),
        'source:population': (str, 'INSEE 2014'),
        'ref:INSEE': (convert_to_int, 31291)},
    u'Colomiers': {
        'population': (convert_to_int, 38541),
        'source:population': (str, 'INSEE 2014'),
        'ref:INSEE': (convert_to_int, 31149)},
    u'Plaisance-du-Touch': {
        'population': (convert_to_int, 17278),
        'source:population': (str, 'INSEE 2014'),
        'ref:INSEE': (convert_to_int, 31424)},
    u'Cugnaux': {
        'population': (convert_to_int, 17004),
        'source:population': (str, 'INSEE 2014'),
        'ref:INSEE': (convert_to_int, 31157)},
    u'Pibrac': {
        'population': (convert_to_int, 8226),
        'source:population': (str, 'INSEE 2014'),
        'ref:INSEE': (convert_to_int, 31417)},
    u'Toulouse': {
        'population': (convert_to_int, 466297),
        'source:population': (str, 'INSEE 2014'),
        'ref:INSEE': (convert_to_int, 31555)},       
}

Let's create an audit class for accuracy. It will compare each information from items having a "population" tag to the standard above.

In [16]:
from accuracy_audit import DataAccuracyAudit

In [17]:
#Parse and audit
audit = [DataAccuracyAudit(gold_standard_insee)]
%time nonconformities = parse_and_audit(dataset_path, audit)
display(HTML(tabulate.tabulate(nonconformities, tablefmt='html')))

CPU times: user 6.92 s, sys: 0 ns, total: 6.92 s
Wall time: 6.92 s


Accuracy,"""INSEE 2013"" value provided for ""source:population"" of Plaisance-du-Touch is inaccurate. Expected value is ""INSEE 2014""."
Accuracy,"""16091"" value provided for ""population"" of Plaisance-du-Touch is inaccurate. Expected value is ""17278""."
Accuracy,"""INSEE 2013"" value provided for ""source:population"" of Colomiers is inaccurate. Expected value is ""INSEE 2014""."
Accuracy,"""35186"" value provided for ""population"" of Colomiers is inaccurate. Expected value is ""38541""."
Accuracy,"""INSEE 2013"" value provided for ""source:population"" of Toulouse is inaccurate. Expected value is ""INSEE 2014""."
Accuracy,"""441802"" value provided for ""population"" of Toulouse is inaccurate. Expected value is ""466297""."
Accuracy,"""INSEE 2013"" value provided for ""source:population"" of Pibrac is inaccurate. Expected value is ""INSEE 2014""."
Accuracy,"""8091"" value provided for ""population"" of Pibrac is inaccurate. Expected value is ""8226""."


Some accuracy issues are reported because data in OpenStreetMap is not up to date with census of 2014.
No issue is reported for Tournefeuille because I have manually updated the OpenStreetMap database.

There are many more accuracy checks that we can do. For example, building with commercial activities have their phone number and web site mentioned in the OpenStreetMap database. Accuracy would have been assessed by checking existence of web site or by comparing phone number to official records.

<a id='Data completeness'/>

### Completeness *[audit](#Data quality)*

Assessing completeness of data is a difficult task. We'll do the work for pharmacies. We will use another standard: [Pages Jaunes](https://www.pagesjaunes.fr/annuaire/tournefeuille-31/pharmacies). Pages Jaunes provides the same kind of services than Yellow Pages.

Here is the list of pharmacies we expect to find:

In [18]:
gold_standard_pages_jaunes = [
    (u'Pharmacie Denise Ribère', (u'2', u'Rue Platanes', 31170, u'Tournefeuille')),
    (u'Pharmacie De La Ramée', (u'102', u'Chemin Larramet', 31170, u'Tournfeuille')),
    (u'Pharmacie Cap 2000', (u'1', u'Boulevard Jean Gay', 31170, u'Tournfeuille')),
    (u'Pharmacie De La Commanderie', (u'110', u'Avenue Marquisat', 31170, u'Tournfeuille')),
    (u'Pharmacie Julien Riviére-Sacaze', (u'18', u'Boulevard Eugène Montel', 31170, u'Tournfeuille')),
    (u'Pharmacie Arc En Ciel', (u'19', u'Avenue Alphonse Daudet', 31170, u'Tournfeuille')),
    (u'Pharmacie Du Centre', (u'67', u'Rue Gaston Doumergue', 31170, u'Tournfeuille')),
    (u'La Pharmacie Du Vieux Pigeonnier', (u'3', u'Rue Hector Berlioz', 31170, u'Tournfeuille')),
    (u'Pharmacie De Pahin', (u'37', u'Chemin Fournaulis', 31170, u'Tournfeuille'))]

Let's create an audit class for completeness. It will compare each information from items having a "amenity/pharmacy" tag to the standard above.

In [19]:
from completeness_audit import DataCompletenessAudit

In [20]:
#Parse and audit
audit = [DataCompletenessAudit(gold_standard_pages_jaunes, warnings=True)]
%time nonconformities = parse_and_audit(dataset_path, audit)
display(HTML(tabulate.tabulate(nonconformities, tablefmt='html')))

CPU times: user 6.74 s, sys: 12 ms, total: 6.75 s
Wall time: 6.75 s


Warning,"Pharmacy ""Pharmacie Saint-Simon"" found but not expected."
Warning,"Pharmacy ""Pharmacie de Lardenne"" found but not expected."
Warning,"Pharmacy ""Pharmacie du Touch"" found but not expected."
Warning,"Pharmacy ""Pharmacie Bruna Rosso"" found but not expected."
Warning,"Pharmacy ""Pharmacie des Marots"" found but not expected."
Warning,"Pharmacy ""Pharmacie Édourad Cabane"" found but not expected."
Warning,"Pharmacy ""Pharmacie de la Paderne"" found but not expected."
Warning,"Pharmacy ""Pharmacie des Jasmins"" found but not expected."
Warning,"Pharmacy ""Pharmacie des Pradettes"" found but not expected."
Warning,"Pharmacy ""Pharmaccie Arc-en-Ciel"" found but not expected."
Warning,"Pharmacy ""Pharmacie Robin"" found but not expected."


4 pharmacies are reported as missing. Some others are reported as present but not expected. This is because dataset extends over boundary of Tournefeuille. We can notice two things:

- Pharmacie Denise Ribère is missing and Pharmacie Ribère has been found. It may be the pharmacy we expected.
- Pharmacie Arc En Ciel is missing and Pharmac**c**ie Arc-en-Ciel has been found. Our string comparison function convert to lower case and replace - by space, but there is a typo in OpenStreetMap database. Use of [fuzzy string](https://streamhacker.com/2011/10/31/fuzzy-string-matching-python/) matching algorithms might have helped in this situation.

For the last two missing items (La Pharmacie Du Vieux Pigeonnier and Pharmacie Julien Riviére-Sacaze), it seems at first glance that dataset is simply uncomplete. But after a closer look to Pages Jaunes on the location of pharmacies, it seems that they match position of Pharmacie Hebraud Meneghetti and Pharmacie Robin.

A simple rename of pharmacies in OpenStreetMap dataset would be enought to ensure a 100% completeness.

<a id='Data consistency'/>

### Consistency *[audit](#Data quality)*

Consistency audit consits in finding contradictory information in the dataset or find issues that prevent us from using some information in the dataset.

In a previous [section](#XML data structure), we have seen that **relation** elements refer to **node**, **way** or other **relation** through the **member** element. Similarly, **way** elements refer to nodes throught **nd** item.

A consistent dataset would provide **relation** and **way** pointing to **node** and **way** also present in the dataset. This is the check we are going to implement:

In [21]:
from consistency_audit import DataConsistencyAudit

In [22]:
#Parse and audit
audit = [DataConsistencyAudit()]
%time nonconformities = parse_and_audit(dataset_path, audit)
display(HTML(tabulate.tabulate(nonconformities, tablefmt='html')))

CPU times: user 9.6 s, sys: 24 ms, total: 9.62 s
Wall time: 9.61 s


Consistency,635 ways refer to non-present entities out of 71960 (0.9%)
Consistency,145 relations refer to non-present entities out of 469 (30.9%)


Some ways (a very low percentage) refer to non-present nodes. A significant number of relations refer to non-present entities. One possible explanation is that towns may be not completly extracted and relation defines town boundaries with nodes or ways. Those nodes or ways are missing because they are out of the box we have extracted from OpenStreetMap.

<a id='Data uniformity'/>

### Uniformity *[audit](#Data quality)*

To audit uniformity, we are going to focus on the way addresses are provided in the dataset.

Here is an example:

```xml
<relation id="1246249" version="2" timestamp="2017-08-21T10:30:22Z" changeset="51299391" uid="922338" user="Hervé TUC">
    <member type="way" ref="74688949" role="outer"/>
    <member type="way" ref="74695300" role="outer"/>
    <member type="way" ref="74692941" role="outer"/>
    <member type="way" ref="74688530" role="outer"/>
    <tag k="addr:city" v="Toulouse"/>
    <tag k="addr:housenumber" v="42"/>
    <tag k="addr:postcode" v="31057"/>
    <tag k="addr:street" v="Avenue Gaspard Coriolis"/>
    ...
```

The item (a relation here) is documented with tags addr:city, addr:housenumber, addr:postcode, addr:street. The addresses in the dataset will be considered uniform if each of them contain all those components. In addition, the way addr:street are recorded will be analyzed to check if mulitple ways of writing [Rue, Avenue, Boulevard, Place, ...] are used. The audit class will report any non-uniformity throughout the dataset:

In [23]:
from uniformity_audit import DataUniformityAudit

In [24]:
#Parse and audit
audit = [DataUniformityAudit(warnings=False)]
%time nonconformities = parse_and_audit(dataset_path, audit)
display(HTML(tabulate.tabulate(nonconformities, tablefmt='html')))
streets_patterns = audit[0].getStreetsPatterns()
print streets_patterns

CPU times: user 7.71 s, sys: 32 ms, total: 7.74 s
Wall time: 7.73 s


Uniformity,"2318 elements miss the following fields: city, postcode."
Uniformity,2210 elements miss the following fields: postcode.
Uniformity,59 elements miss the following fields: city.
Uniformity,8 elements miss the following fields: housenumber.
Uniformity,"12 elements miss the following fields: city, postcode, housenumber."
Uniformity,"13 elements miss the following fields: postcode, housenumber."
Uniformity,"11 elements miss the following fields: city, housenumber."


set([u'Boulevard', u'Rond-Point', u'rue', u'impasse', u'avenue', u'Route', u'Passage', u'Impasse', u'Av.', u'Chemin', u'place', u'Clos', u'Place', u'all\xe9e', u'Rue', u'Avenue', u'All\xe9e'])


In terms of uniformity of providing the same address components, the most common issue is to not have postcode and city. Few times housenumber is also missing. Fixing housenumber automatically seems difficult. Fixing postcode may be easy in the case city, as recorded in OpenStreetMap has a single postcode and item has a city attached. There is nothing obvious we can do for items having no postcode and city fields. One possible solution would be to check inclusion of node (given its latitude / longitude) in the polygon delimiting city (as provided by **relation** elements) which can be solved with a little [maths](http://geomalgorithms.com/a03-_inclusion.html).

There is also a lack of uniformity in the way streets are recorded. For example we can see Av. avenue, or Avenue but this is not a big deal and be fixed easilly.

<a id='Audit conclusion'/>

### Conclusion *[audit](#Data quality)*

The audit performed is rather incomplete in terms of check that can be performed. But we have seen how we can audit any kind of nonconformity (validity, accuracy, completeness, consistency and uniformity).

Yet, the frontier between each type of audit may be tenuous:
- the completeness issues indentified with missing pharmacies turned out to be an accuracy problem (pharmacies are in the dataset but with a different name).
- the inconsistency issue with nodes or ways referenced either in ways or relations but missing from dataset may be seen as a completeness issue. Nodes and ways probably exist in the full OpenStreetMap database, so this is probably more related to how the data from full database is extracted with a box selection.
- the uniformity issues in the way addresses are recorded may also be seen as a completeness issue because housenumbers are missing.

The classification of nonconformities is not that important, but the list (validity, accuracy, completeness, consistency and uniformity) is probably a good hint to be sure we do not forget some kind of checks.

On large datasets, it seems not impossible but very tedious to run a full quality audit. Knowing the scope of analysis helps in selecting the minimum set of audits to run on the dataset.

Using an iterative parser is clearly an additional difficulty in writing audit code, things would have been much simpler by using a full-parser like the ones provided by [lxml](http://lxml.de/) library. lxml also implements pretty advanced XPath requests that would have made both auditing and cleaning much faster.

The following code wraps all audit tasks and returns a table with all kind of nonconformities:

In [25]:
#Parse and audit
full_audit = [DataValidityAudit(schema), DataAccuracyAudit(gold_standard_insee), 
         DataCompletenessAudit(gold_standard_pages_jaunes), DataConsistencyAudit(), DataUniformityAudit()]
%time nonconformities = parse_and_audit(dataset_path, full_audit)
display(HTML(tabulate.tabulate(nonconformities, tablefmt='html')))

CPU times: user 22.5 s, sys: 44 ms, total: 22.6 s
Wall time: 22.6 s


Accuracy,"""INSEE 2013"" value provided for ""source:population"" of Plaisance-du-Touch is inaccurate. Expected value is ""INSEE 2014""."
Accuracy,"""16091"" value provided for ""population"" of Plaisance-du-Touch is inaccurate. Expected value is ""17278""."
Accuracy,"""INSEE 2013"" value provided for ""source:population"" of Colomiers is inaccurate. Expected value is ""INSEE 2014""."
Accuracy,"""35186"" value provided for ""population"" of Colomiers is inaccurate. Expected value is ""38541""."
Accuracy,"""INSEE 2013"" value provided for ""source:population"" of Toulouse is inaccurate. Expected value is ""INSEE 2014""."
Accuracy,"""441802"" value provided for ""population"" of Toulouse is inaccurate. Expected value is ""466297""."
Accuracy,"""INSEE 2013"" value provided for ""source:population"" of Pibrac is inaccurate. Expected value is ""INSEE 2014""."
Accuracy,"""8091"" value provided for ""population"" of Pibrac is inaccurate. Expected value is ""8226""."
Completeness,"Pharmacy ""Pharmacie Denise Ribère"" is missing in dataset."
Completeness,"Pharmacy ""Pharmacie Julien Riviére-Sacaze"" is missing in dataset."
Completeness,"Pharmacy ""Pharmacie Arc En Ciel"" is missing in dataset."


Note that we have not found any validity issue. The XML structure is pretty simple and OpenStreetMap provides XML files that can be trusted in terms of structure. The issues come from data that has been provided by users.

In the [next section](#Data cleaning), we are going to clean this dataset before importing it into a database.

<a id='Data cleaning'/>

## Data cleaning *[top](#Top)*

<a id='Method'/>

### Method *[cleaning](#Data cleaning)*

We are not going to clean the XML file we have downloaded from OpenStreetMap. As we need to parse it iteratively, writing a cleaning algorithm would be pretty difficult.

Here are the steps we are going to follow:

- Export dataset into a Python dictionnary-like structure
- Clean the dictionnary
- Save dictionnary into a JSON file

**Note:** exporting the dataset to a dictionnary may be memory consumming. I have not used any technique here to reduce memory footprint but [shelve](https://docs.python.org/2/library/shelve.html?highlight=shelve#module-shelve) in standard library may help. Alternatives may be [diskcache](http://www.grantjenks.com/docs/diskcache/tutorial.html) and even directly [pymongo](https://api.mongodb.com/python/current/tutorial.html).

<a id='Converting to dictionnary-like structure'/>

### Converting to dictionnary-like structure *[cleaning](#Data cleaning)*

We are going to use a similar technique than the one we had for data quality audit. We are basically going to plug a callback class to the SAX content handler to load OpenStreetMap dataset into a dictonnary-like structure.

First we need to define the dictionnary structure we want to have:

```python
schema_dict = {
    'nodes': [
        {'osmid': 0, #the OpenStreetMap id of node
         'latitude': 0., #latitude [-90, 90]
         'longitude': 0., #longitude [-180, 180]
         'userid': 0, #OpenStreetMap id of owner
         'tags': [ #list of associated tags
             {'key': '', #tag key
              'value': ''}] #tag value
        }
    ]
    'ways': [
        {'osmid': 0, #the OpenStreetMap id of node
         'userid': 0, #OpenStreetMap id of owner
         'tags': [ #list of associated tags
             {'key': '', #tag key
              'value': ''}] #tag value
         'nodes': [ ] #a list of nodes osmid
        }
    ]
    'relations': [
        {'osmid': 0, #the OpenStreetMap id of node
         'userid': 0, #OpenStreetMap id of owner
         'tags': [ #list of associated tags
             {'key': '', #tag key
              'value': ''}] #tag value
         'nodes': [ ] #a list of nodes osmid
         'ways': [ ] #a list of ways osmid
         'relations': [ ] #a list of relations osmid
        }
    ]
```

In [26]:
from dictionnary_export import DictionnaryExport

In [27]:
import sys

#Parse and extract
parser = xml.sax.make_parser()
dataset_dict = { }
extract_class = DictionnaryExport(dataset_dict)
with OpenStreetMapXmlHandler() as handler:
    handler.registerStartEventCallback(extract_class.startEventCallback)
    handler.registerEndEventCallback(extract_class.endEventCallback)
    parser.setContentHandler(handler)
    %time parser.parse(dataset_path)
for kind in ['nodes', 'ways', 'relations']:
    print "{} {} exported.".format(len(dataset_dict[kind]), kind)

CPU times: user 10.2 s, sys: 68 ms, total: 10.3 s
Wall time: 10.4 s
428787 nodes exported.
71960 ways exported.
469 relations exported.


All nodes, ways and relations have been exported. We can investigate few items:

In [28]:
print dataset_dict['nodes'][-1]

{'latitude': 43.5874494, 'userid': 568960, 'osmid': 5053431726, 'longitude': 1.349595, 'tags': [{'value': u'bicycle_parking', 'key': u'amenity'}]}


In [29]:
print dataset_dict['ways'][-1]

{'nodes': [5052004973, 5052004972, 5052004971, 5052004970, 5052004973], 'userid': 568960, 'osmid': 517760389, 'tags': [{'value': u'parking', 'key': u'amenity'}]}


In [30]:
print dataset_dict['relations'][-1]

{'osmid': 7494469, 'tags': [{'value': u'cemetery', 'key': u'landuse'}, {'value': u'Cimeti\xe8re de Lardenne', 'key': u'name'}, {'value': u'multipolygon', 'key': u'type'}], 'ways': [24788185, 167601981], 'userid': 922338, 'relations': [], 'nodes': []}


Having the dataset in this format is much more handy. But making queries into it requires to write dedicated functions. That's all the point to use a database: anything to perform requests is already implemented for us. For now, it is convenient enough to perform the data cleaning.

<a id='Cleaning accuracy issues'/>

### Cleaning accuracy issues *[cleaning](#Data cleaning)*

In data accuracy audit [section](#Data accuracy) we have spotted few accuracy issues regarging city populations. The idea here is just to update nodes and relations having population tag so thet match the INSEE standard.

A simple function may be written for that purpose:

In [31]:
#Clean accuracy issues
def clean_accuracy(dataset, standard):
    """
    Clean accuracy issues according to INSEE standard.
    
    The dataset is modified in-place.
    
    - dataset: dataset to be cleaned
    - standard: gold standard for accuracy criteria (INSEE gold standard)
    - return: list of updated nodes index and list of updated relations index
    """
    nodes = [ ]
    relations = [ ]
    def process_item(kind, modified):
        #Process nodes
        for iitem, item in enumerate(dataset[kind]):
            tag_keys = [tag['key'] for tag in item['tags']]
            #Has population tag?
            try:
                population_index = tag_keys.index('population')
            except ValueError:
                continue
            #Update
            try:
                city = item['tags'][tag_keys.index('name:fr')]['value']
            except ValueError:
                city = item['tags'][tag_keys.index('name')]['value']
            source_population_index = tag_keys.index('source:population')
            if item['tags'][source_population_index]['value'] != standard[city]['source:population'][1]:
                item['tags'][source_population_index]['value'] = standard[city]['source:population'][1]
                item['tags'][population_index]['value'] = standard[city]['population'][1]
                print "updated {} {} ({}).".format(kind.rstrip('s'), item["osmid"], city)
                modified.append(iitem)
            else:
                print "{} {} ({}) is left unchanged.".format(kind.rstrip('s'), item["osmid"], city)
    #Process nodes
    process_item("nodes", nodes)
    #Process relations
    process_item("relations", relations)    
    return nodes, relations

In [32]:
%time inodes, irelations = clean_accuracy(dataset_dict, gold_standard_insee)
for inode in inodes:
    pprint(dataset_dict["nodes"][inode])
for irelation in irelations:
    pprint(dataset_dict["relations"][irelation])

node 26691412 (Tournefeuille) is left unchanged.
updated node 26691742 (Plaisance-du-Touch).
updated relation 35078 (Colomiers).
updated relation 35738 (Toulouse).
updated relation 158881 (Pibrac).
CPU times: user 404 ms, sys: 44 ms, total: 448 ms
Wall time: 374 ms
{'latitude': 43.5653012,
 'longitude': 1.2978343,
 'osmid': 26691742,
 'tags': [{'key': u'addr:postcode', 'value': u'31830'},
          {'key': u'name', 'value': u'Plaisance-du-Touch'},
          {'key': u'name:fr', 'value': u'Plaisance-du-Touch'},
          {'key': u'name:oc', 'value': u'Plasen\xe7a-deu-Toish'},
          {'key': u'place', 'value': u'town'},
          {'key': u'population', 'value': 17278},
          {'key': u'ref:FR:SIREN', 'value': u'213104243'},
          {'key': u'ref:INSEE', 'value': u'31424'},
          {'key': u'source:population', 'value': 'INSEE 2014'},
          {'key': u'wikidata', 'value': u'Q608231'}],
 'userid': 3057995}
{'nodes': [26692194],
 'osmid': 35078,
 'relations': [],
 'tags': [{'key'

<a id='Cleaning completeness issues'/>

### Cleaning completeness issues *[cleaning](#Data cleaning)*

In data completeness audit [section](#Data completeness) we have indentified missing records in the dataset. It turned out that issues indentified as completeness issues may be seen as accuracy issues since the pharmacies are not  recorded with the proper name.

We can deal with these issues with a simple function:

In [33]:
#Clean completeness issues
def clean_completeness(dataset, mapping):
    """
    Clean completeness issues according to Pages Jaunes standard.
    
    The dataset is modified in-place.
    
    - dataset: dataset to be cleaned
    - mapping: mapping of phamarcy names
    - return: list of updated nodes index
    """
    nodes = [ ]
    for inode, node in enumerate(dataset["nodes"]):
        tag_keys = [tag['key'] for tag in node['tags']]
        #Has amenity tag?
        try:
            amenity_index = tag_keys.index('amenity')
        except ValueError:
            continue
        else:
            #Check this is a pharmacy
            if node["tags"][amenity_index]['value'].lower() != "pharmacy":
                continue
        #Fix pharmacy name
        name_index = tag_keys.index('name')
        name = node["tags"][name_index]["value"]
        try:
            fixed_name = mapping[name]
        except KeyError:
            print u"{} is left unchanged.".format(name)
        else:
            node["tags"][name_index]["value"] = fixed_name
            nodes.append(inode)
            print u"{} has been updated to {}.".format(name, fixed_name)
    return nodes

In [34]:
pharmacy_mapping = {u"Pharmacie Ribère": u"Pharmacie Denise Ribère", 
                    u"Pharmaccie Arc-en-Ciel": u"Pharmacie Arc En Ciel", 
                    u"Pharmacie Robin": u"Pharmacie Julien Riviére-Sacaze",
                    u"Pharmacie Hebraud Meneghetti": u"La Pharmacie Du Vieux Pigeonnier",
                    u"Pharmacie de la Ramée": u"Pharmacie De La Ramée",
                    u"Pharmacie de la Commanderie": u"Pharmacie De La Commanderie",
                    u"Pharmacie du Centre": u"Pharmacie Du Centre",
                    u"Pharmacie de Pahin": "Pharmacie De Pahin",
                    u"Pharmacie CAP 2000": u"Pharmacie Cap 2000"}
%time inodes = clean_completeness(dataset_dict, pharmacy_mapping)
for inode in inodes:
    pprint(dataset_dict["nodes"][inode])

Pharmacie Saint-Simon is left unchanged.
Pharmacie de Lardenne is left unchanged.
Pharmacie de Pahin has been updated to Pharmacie De Pahin.
Pharmacie du Touch is left unchanged.
Pharmacie Bruna Rosso is left unchanged.
Pharmacie des Marots is left unchanged.
Pharmacie Édourad Cabane is left unchanged.
Pharmacie de la Paderne is left unchanged.
Pharmacie des Jasmins is left unchanged.
Pharmacie des Pradettes is left unchanged.
Pharmaccie Arc-en-Ciel has been updated to Pharmacie Arc En Ciel.
Pharmacie Robin has been updated to Pharmacie Julien Riviére-Sacaze.
Pharmacie Ribère has been updated to Pharmacie Denise Ribère.
Pharmacie des Tibaous is left unchanged.
Pharmacie Hebraud Meneghetti has been updated to La Pharmacie Du Vieux Pigeonnier.
Pharmacie de la Commanderie has been updated to Pharmacie De La Commanderie.
Pharmacie de la Ramée has been updated to Pharmacie De La Ramée.
Pharmacie CAP 2000 has been updated to Pharmacie Cap 2000.
Pharmacie Sol is left unchanged.
Pharmacie du C

<a id='Cleaning consistency issues'/>

### Cleaning consistency issues *[cleaning](#Data cleaning)*

Some **node**s, **way**s or **relation**s are referenced in **way** or **relation** items but are missing in dataset we have extracted. I have decided to remove any of those items from the dictionnary-like dataset in order to keep a consistent database as output.

The audit class can be requested to get a set of missing items. Knowing the list, we just have to remove any reference to those items in our cleaned dataset. This can be done with the following function:

In [35]:
#Clean consistency issues
def clean_consistency(dataset, missing_nodes, missing_ways, missing_relations):
    """
    Clean consistency issues by removing any reference to missing nodes, ways or relations.
    
    The dataset is modified in-place.
    
    - dataset: dataset to be cleaned
    - missing_nodes: set of missing nodes
    - missing_ways: set of missing ways
    - missing_relations: set of missing relations
    - return: set of updated ways index, set of updated relations
    """
    ways = set()
    relations = set()
    #Process ways
    for iway, way in enumerate(dataset["ways"]):
        to_be_removed = [ ]
        for inode, node in enumerate(way["nodes"]):
            if node in missing_nodes:
                to_be_removed.append(inode)
        to_be_removed.reverse() #so we do not change indexes of items in list while we iterate
        for i in to_be_removed:
            value = way["nodes"].pop(i)
            assert value in missing_nodes #cross-check, just in case I introduce errors by mistake
            ways.add(iway)
    print "{} ways updated.".format(len(ways))
    #Process relations
    for irelation, relation in enumerate(dataset["relations"]):
        for (kind, missing) in [("nodes", missing_nodes), ("ways", missing_ways), ("relations", missing_relations)]:
            to_be_removed = [ ]
            for iitem, item in enumerate(relation[kind]):
                if item in missing:
                    to_be_removed.append(iitem)
            to_be_removed.reverse() #so we do not change indexes of items in list while we iterate
            for i in to_be_removed:
                value = relation[kind].pop(i)
                assert value in missing #cross-check, just in case I introduce errors by mistake
                relations.add(irelation)
    print "{} relations updated.".format(len(relations))
    return ways, relations

In [36]:
missing_nodes = full_audit[3].getMissingNodes()
missing_ways = full_audit[3].getMissingWays()
missing_relations = full_audit[3].getMissingRelations()
%time iways, irelations = clean_consistency(dataset_dict, missing_nodes, missing_ways, missing_relations)

635 ways updated.
145 relations updated.
CPU times: user 96 ms, sys: 24 ms, total: 120 ms
Wall time: 75.9 ms


The number of updated ways and relations matches the number of issues we have identified earlier.

<a id='Cleaning uniformity issues'/>

### Clean uniformity issues *[cleaning](#Data cleaning)*

In data quality audit [section](#Data uniformity) we have identified two different kinds of nonconformities:
- missing addr:housenumber or addr:postcode
- non-uniform way of naming streets component

We will not fix the addr:housenumber because there is no obvious way to do it. We'll try to fix addr:postcode as we may have postcode information in city data. We may not be able to fix 100% of postcode though:
- some big cities have multiple postcodes
- the city in addr:city may not match the city name

Finally, we can easilly solve the second kind of non-uniformity by providing a mapping.

The following function performs all cleaning related to uniformity issues:

In [37]:
#Clean uniformity issues
def clean_uniformity(dataset, street_mapping):
    """
    Clean uniformity issues by:
    - replacing street components by uniform ones
    - trying to set postcode automatically
    
    The dataset is modified in-place.
    
    - dataset: dataset to be cleaned
    - street_mapping: mapping of street components
    - return: set of updated nodes, set of updated ways index, set of updated relations
    """
    nodes = set()
    ways = set()
    relations = set()
    #Process street components
    def process_street_component(kind, dataset, mapping, updated):
        for iitem, item in enumerate(dataset[kind]):
            for tag in item['tags']:
                if tag['key'] == 'addr:street':    
                    component = tag['value'].strip().split()[0]
                    try:
                        fixed_component = mapping[component]
                    except KeyError:
                        continue
                    else:
                        tag['value'] = tag['value'].replace(component, fixed_component, 1)
                        updated.add(iitem)
    for kind, mapping, updated in [("nodes", street_mapping, nodes),
                                   ("ways", street_mapping, ways),
                                   ("relations", street_mapping, relations)]:
        process_street_component(kind, dataset, mapping, updated)
    #Process post codes
    #First we need to get a mapping of city with postcode
    city_postcodes = { }
    def get_postcodes(kind, dataset, postcodes):
        for iitem, item in enumerate(dataset[kind]):
            tag_keys = [tag['key'] for tag in item['tags']]
            try:
                population_index = tag_keys.index('ref:INSEE')
            except ValueError:
                continue
            else:
                #This is a city
                try:
                    city = item['tags'][tag_keys.index('name:fr')]['value']
                except ValueError:
                    city = item['tags'][tag_keys.index('name')]['value']
                #Get postcode
                try:
                    postcode = item['tags'][tag_keys.index('addr:postcode')]['value']
                except:
                    pass
                else:
                    if postcode.isdigit():
                        postcodes[city] = postcode
                    else:
                        print "Items in {} cannot be fixed because there are many postcodes: {}.".format(
                                city, postcode)
    for kind in ["nodes", "relations"]:
        get_postcodes(kind, dataset, city_postcodes)
    #Now we can try to fix addresses for which postcode is missing
    def update_postcodes(kind, dataset, postcodes, updated):
        for iitem, item in enumerate(dataset[kind]):
            tag_keys = [tag['key'] for tag in item['tags']]
            try:
                street_index = tag_keys.index('addr:street')
            except ValueError:
                continue
            else:
                try:
                    postcode_index = tag_keys.index('addr:postcode')
                except ValueError:
                    #postcode is not set
                    try:
                        city = item["tags"][tag_keys.index('addr:city')]["value"]
                    except ValueError:
                        #Item has neither postcode nor city...
                        continue
                    try:
                        postcode = postcodes[city]
                    except KeyError:
                        #We have no postcode for this city
                        #print "{} with osmid {} cannot be fixed because there is no postcode for city {}.".format(
                        #        kind, item['osmid'], city)
                        continue
                    else:
                        item["tags"].append({"key": "addr:postcode", "value": postcode})
                        #print "{} with osmid {} now has a addr:postcode {}.".format(
                        #        kind, item['osmid'], postcode)
                        updated.add(iitem)
                else:
                    #postcode is already set
                    continue
    for kind, updated in [("nodes", nodes),
                          ("ways", ways),
                          ("relations", relations)]:
        update_postcodes(kind, dataset, city_postcodes, updated)   
    print "{} nodes updated.".format(len(nodes))
    print "{} ways updated.".format(len(ways))
    print "{} relations updated.".format(len(relations))    
    return nodes, ways, relations

In [38]:
#Reminder of all patterns encountered
print streets_patterns

set([u'Boulevard', u'Rond-Point', u'rue', u'impasse', u'avenue', u'Route', u'Passage', u'Impasse', u'Av.', u'Chemin', u'place', u'Clos', u'Place', u'all\xe9e', u'Rue', u'Avenue', u'All\xe9e'])


In [39]:
#Mapping
street_mapping = {u"rue": u"Rue",
                  u"impasse": u"Impasse",
                  u"avenue": u"Avenue",
                  u"Av.": u"Avenue",
                  u"place": u"Place",
                  u"allée": u"Allée"}
%time inodes, iways, irelations = clean_uniformity(dataset_dict, street_mapping)

Items in Toulouse cannot be fixed because there are many postcodes: 31000;31100;31200;31300;31400;31500.
590 nodes updated.
2 ways updated.
0 relations updated.
CPU times: user 1.01 s, sys: 44 ms, total: 1.06 s
Wall time: 945 ms


We have been able to fix few postcodes issues (all for which city was provided and was different from Toulouse).

<a id='Cleaning conclusion'/>

### Conclusion *[cleaning](#Data cleaning)*

Cleaning is far from being perfect, but we have illustrated different techniques to clean a dataset. This is definitely a time-consumming activity :)

It's now time to export dataset into files and databases.

<a id='Data export'/>

## Data export *[top](#Top)*

<a id='JSON MongoDB'/>

### To JSON and MongoDB *[export](#Data export)*

This is the couple I am least confortable with. Luckily, our dictionnary-like structure is really adapted to both transformation to JSON file or for mass import into MongoDB.

Let's start with dumping a JSON file:

In [40]:
import json
with open('data.json', 'w') as fobj:
    %time fobj.write(json.dumps(dataset_dict))
print "Size of JSON file {} bytes.".format(os.path.getsize('data.json'))

CPU times: user 824 ms, sys: 104 ms, total: 928 ms
Wall time: 950 ms
Size of JSON file 68692676 bytes.


Writing is pretty fast for a 68 Mb file.

Let's try to reload the file:

In [41]:
with open('data.json', 'r') as fobj:
    %time dataset_dict = json.loads(fobj.read())
print "Dataset with {} nodes, {} ways and {} relations.".format(len(dataset_dict["nodes"]), 
        len(dataset_dict["ways"]), len(dataset_dict["relations"]))

CPU times: user 2.07 s, sys: 92 ms, total: 2.16 s
Wall time: 2.17 s
Dataset with 428787 nodes, 71960 ways and 469 relations.


We have not lost anything. Reading is a JSON is longer than writing it though. We can now store the data into a MongoDB database:

In [42]:
#Connect to MongoDB and remove any previous database (if any)
from pymongo import MongoClient
mongodb_client = MongoClient()
mongodb_client.drop_database('udacity-wrangling')

In [45]:
#Mass import from JSON file of documents
db = mongodb_client['udacity-wrangling']
nodes = db['nodes']
nodes.insert_many(dataset_dict["nodes"])
ways = db['ways']
ways.insert_many(dataset_dict["ways"])
relations = db["relations"]
relations.insert_many(dataset_dict["relations"])
print db.collection_names()

[u'nodes', u'system.indexes', u'ways', u'relations']


In [46]:
#What's in there?
print nodes.find_one()
print ways.find_one()
print relations.find_one()

{u'osmid': 8138771, u'tags': [{u'value': u'traffic_signals', u'key': u'highway'}], u'userid': 134812, u'longitude': 1.3904169, u'latitude': 43.5618389, u'_id': ObjectId('59a0bb83c8f807671d8c09aa')}
{u'nodes': [26554082, 26554083, 26554084, 26554085, 26554086, 26554087, 26554088, 26554089, 26554090, 26554091, 26554092, 26554093, 26554094, 492014327, 492014294, 26554096, 26554097, 492005005, 492005006, 26554098, 26554099, 26554100, 26554101, 26554102, 26554103, 1824910001, 1824910003, 26554104, 1824910007, 26554105, 1824910010, 26554106, 26554107, 26554108, 26554079, 26554080, 26554082], u'_id': ObjectId('59a0bb8ac8f807671d92949d'), u'userid': 125897, u'osmid': 4357819, u'tags': []}
{u'osmid': 13551, u'tags': [{u'value': u'Rue des Catalpas', u'key': u'name'}, {u'value': u'associatedStreet', u'key': u'type'}], u'ways': [6228254], u'userid': 722137, u'relations': [], u'nodes': [265545746, 265545747, 265545748, 265546434, 265546435, 265546436], u'_id': ObjectId('59a0bb8bc8f807671d93adb5')}


In [47]:
#Request by OpenStreetMap id:
for item in nodes.find({'osmid': 8138771}):
    pprint(item)

{u'_id': ObjectId('59a0bb83c8f807671d8c09aa'),
 u'latitude': 43.5618389,
 u'longitude': 1.3904169,
 u'osmid': 8138771,
 u'tags': [{u'key': u'highway', u'value': u'traffic_signals'}],
 u'userid': 134812}


In [48]:
#Request east-most nodes (max 3 nodes are returned), SQL LIMIT equivalent
for item in nodes.find({'longitude': {'$gt': 1.39}}).limit(3):
    pprint(item)

{u'_id': ObjectId('59a0bb83c8f807671d8c09aa'),
 u'latitude': 43.5618389,
 u'longitude': 1.3904169,
 u'osmid': 8138771,
 u'tags': [{u'key': u'highway', u'value': u'traffic_signals'}],
 u'userid': 134812}
{u'_id': ObjectId('59a0bb83c8f807671d8c09ab'),
 u'latitude': 43.5619922,
 u'longitude': 1.3901005,
 u'osmid': 8138772,
 u'tags': [],
 u'userid': 134812}
{u'_id': ObjectId('59a0bb83c8f807671d8c0c9c'),
 u'latitude': 43.5702514,
 u'longitude': 1.3904678,
 u'osmid': 53869616,
 u'tags': [],
 u'userid': 13384}


In [49]:
#Refined latitude / longitude box (equivalent to City center dataset)
for item in nodes.find({'longitude': {'$gt': 1.3434, '$lt': 1.3496}, 
                        'latitude': {'$gt': 43.5799, '$lt': 43.5838}}).limit(3):
    pprint(item) 

{u'_id': ObjectId('59a0bb83c8f807671d8c0a9b'),
 u'latitude': 43.5827846,
 u'longitude': 1.3466543,
 u'osmid': 26691412,
 u'tags': [{u'key': u'addr:postcode', u'value': u'31170'},
           {u'key': u'name', u'value': u'Tournefeuille'},
           {u'key': u'name:fr', u'value': u'Tournefeuille'},
           {u'key': u'name:oc', u'value': u'Tornafu\xe8lha'},
           {u'key': u'place', u'value': u'town'},
           {u'key': u'population', u'value': u'26674'},
           {u'key': u'ref:FR:SIREN', u'value': u'213105570'},
           {u'key': u'ref:INSEE', u'value': u'31557'},
           {u'key': u'source:population', u'value': u'INSEE 2014'},
           {u'key': u'wikidata', u'value': u'Q328022'},
           {u'key': u'wikipedia', u'value': u'fr:Tournefeuille'}],
 u'userid': 6523296}
{u'_id': ObjectId('59a0bb83c8f807671d8c0b9b'),
 u'latitude': 43.582037,
 u'longitude': 1.3450115,
 u'osmid': 51983918,
 u'tags': [],
 u'userid': 148173}
{u'_id': ObjectId('59a0bb83c8f807671d8c0ba1'),
 u'la

In [50]:
#Find in document attributes (list)
for item in relations.find({"nodes": 265545746}):
    pprint(item) 

{u'_id': ObjectId('59a0bb8bc8f807671d93adb5'),
 u'nodes': [265545746, 265545747, 265545748, 265546434, 265546435, 265546436],
 u'osmid': 13551,
 u'relations': [],
 u'tags': [{u'key': u'name', u'value': u'Rue des Catalpas'},
           {u'key': u'type', u'value': u'associatedStreet'}],
 u'userid': 722137,
 u'ways': [6228254]}


In [51]:
#Find in document attributes (dict) with kind of SQL UNION and $and operator:
filter_tags = lambda x: x["key"] in ('name:fr', 'ref:INSEE', 'population', 'source:population')
city_criteria = {"$and": [{"tags.key": "ref:INSEE"}, {"tags.key": "population"}]}
items = [node for node in nodes.find(city_criteria)]
items.extend([relation for relation in relations.find(city_criteria)])
for item in items:
    pprint(dict((t["key"], t["value"]) for t in filter(filter_tags, item["tags"])))

{u'name:fr': u'Tournefeuille',
 u'population': u'26674',
 u'ref:INSEE': u'31557',
 u'source:population': u'INSEE 2014'}
{u'name:fr': u'Plaisance-du-Touch',
 u'population': 17278,
 u'ref:INSEE': u'31424',
 u'source:population': u'INSEE 2014'}
{u'name:fr': u'Colomiers',
 u'population': 38541,
 u'ref:INSEE': u'31149',
 u'source:population': u'INSEE 2014'}
{u'name:fr': u'Toulouse',
 u'population': 466297,
 u'ref:INSEE': u'31555',
 u'source:population': u'INSEE 2014'}
{u'name:fr': u'Pibrac',
 u'population': 8226,
 u'ref:INSEE': u'31417',
 u'source:population': u'INSEE 2014'}


In [52]:
#Look for pharmacies in Tournefeuille either with city name or postcode, combination of and and or operators:
find_criteria = {"$and": [{"tags.key": "amenity", "tags.value": "pharmacy"}, 
                          {"$or": [{"tags.key": "addr:postcode", "tags.value": "31170"},
                                   {"tags.key": "addr:city", "tags.value": "Tournefeuille"}]}]}
for node in nodes.find(find_criteria):
    pprint(node)

{u'_id': ObjectId('59a0bb86c8f807671d91f5af'),
 u'latitude': 43.5874398,
 u'longitude': 1.3507017,
 u'osmid': 2157269359L,
 u'tags': [{u'key': u'addr:postcode', u'value': u'31170'},
           {u'key': u'addr:street', u'value': u'Boulevard Vincent Auriol'},
           {u'key': u'amenity', u'value': u'pharmacy'},
           {u'key': u'dispensing', u'value': u'yes'},
           {u'key': u'name', u'value': u'Pharmacie Arc En Ciel'},
           {u'key': u'ref:FR:FINESS', u'value': u'310016126'},
           {u'key': u'source', u'value': u'survey;Celtipharm - 10/2014'}],
 u'userid': 1626}


There is a single one pharmacy with either addr:postcode or addr:city attribute set for pharmacies in Tournfeuille.
The cleaning made is insufficiant because we haven't been able to affect postcode or city to nodes missing both of them.

In [53]:
#Get major contributors: usage of aggregation, grouping and sorting (descending)
#We need to build an aggregation pipeline
for item in nodes.aggregate([{"$group": {"_id": "$userid", "count": {"$sum": 1}}}, #group by userid and count
                             {"$project": {"count": { "$multiply": [ "$count", 100. / nodes.count()]}}}, # calculate %
                             {"$sort": {"count": -1}}, #sort by descending order
                             {"$limit": 10}]): #limit to 10 users
    print item
print nodes.count()

{u'count': 68.14245767712174, u'_id': 1685}
{u'count': 8.911184340943171, u'_id': 306172}
{u'count': 4.423175142903119, u'_id': 115737}
{u'count': 2.2099550592718527, u'_id': 148173}
{u'count': 1.865961421405033, u'_id': 1573648}
{u'count': 1.806491334858566, u'_id': 13384}
{u'count': 1.1768080655430317, u'_id': 125897}
{u'count': 0.980906603978199, u'_id': 571625}
{u'count': 0.9356626949977495, u'_id': 1626}
{u'count': 0.9316980225613183, u'_id': 297482}
428787


The user owning the most number of nodes in OpenStreetMap is also owner of more than 68% of all nodes ! Let's find out if we can know more about him:

In [54]:
#More information about userid 1685
for item in ways.find({"userid": 1685, "tags.key": "source"}).limit(1):
    pprint(item)

{u'_id': ObjectId('59a0bb8ac8f807671d929aee'),
 u'nodes': [343600511,
            3222241389L,
            343600512,
            343600507,
            3222241387L,
            343600513,
            3222241386L,
            247420475],
 u'osmid': 30907996,
 u'tags': [{u'key': u'highway', u'value': u'tertiary'},
           {u'key': u'junction', u'value': u'roundabout'},
           {u'key': u'name', u'value': u'Rond-Point Ut\xe9bo'},
           {u'key': u'source',
            u'value': u'cadastre-dgi-fr source : Direction G\xe9n\xe9rale des Imp\xf4ts - Cadastre. Mise \xe0 jour : 2009'}],
 u'userid': 1685}


It seems that he is working in Direction Générale des Impôts (French Tax Directorate) in the land registry office (french word: cadastre). French land registry office seems to use OpenStreetMap ;)

To end this MongoDB capabilities overview, we are trying to do a join to get latitude and longitude of all nodes in the previous way:

In [55]:
#First let's have a look at what unwind operator does. Match operator enables to use a find in an aggregation
for item in ways.aggregate([{"$match": {"osmid": 30907996}},
                            {"$unwind": "$nodes"},
                            {"$limit": 3}]):
    pprint(item)

{u'_id': ObjectId('59a0bb8ac8f807671d929aee'),
 u'nodes': 343600511,
 u'osmid': 30907996,
 u'tags': [{u'key': u'highway', u'value': u'tertiary'},
           {u'key': u'junction', u'value': u'roundabout'},
           {u'key': u'name', u'value': u'Rond-Point Ut\xe9bo'},
           {u'key': u'source',
            u'value': u'cadastre-dgi-fr source : Direction G\xe9n\xe9rale des Imp\xf4ts - Cadastre. Mise \xe0 jour : 2009'}],
 u'userid': 1685}
{u'_id': ObjectId('59a0bb8ac8f807671d929aee'),
 u'nodes': 3222241389L,
 u'osmid': 30907996,
 u'tags': [{u'key': u'highway', u'value': u'tertiary'},
           {u'key': u'junction', u'value': u'roundabout'},
           {u'key': u'name', u'value': u'Rond-Point Ut\xe9bo'},
           {u'key': u'source',
            u'value': u'cadastre-dgi-fr source : Direction G\xe9n\xe9rale des Imp\xf4ts - Cadastre. Mise \xe0 jour : 2009'}],
 u'userid': 1685}
{u'_id': ObjectId('59a0bb8ac8f807671d929aee'),
 u'nodes': 343600512,
 u'osmid': 30907996,
 u'tags': [{u'key': 

unwind operator enables to "deconstruct" an array. In fact we are going to need this operator to perform a join (lookup operator). Lookup performs a left join. The join syntax is:

In [56]:
#Now let's join
if map(lambda x: int(x), mongodb_client.server_info()['version'].split('.')) > (3, 2, 0):
    try:
        for item in ways.aggregate([{"$match": {"osmid": 30907996}},
                                    {"$unwind": "$nodes"},
                                    {"$lookup": {"$from": "nodes", "$localField": "nodes", "$foreignField": "osmid"}}]):
            pprint(item)
    except:
        "This code is untested because I don't have a recent version of MongoDB yet, I need to updgrade."
else:
    print "Your version of Mongodb does not support $lookup operator."

Your version of Mongodb does not support $lookup operator.


I am using a LTS version of Ubuntu 16.04 coming with MongoDB 2.6 not supporting yet $lookup operator (supported from version 3.2).

MongoDB differs from what I know about SQL:
- no need of schema, it's very esay to store collections from Python
- aggregate, group, sort, limit, etc... works a bit differently than SQL equivalent but product is well documented and it's easy to find answers on [StackOverflow](https://stackoverflow.com/questions/tagged/mongodb) (as always...)
- "documents" are also easier to read, information is not spread into multiple tables
- joining is a bit trickier than SQL, but that's probably the price to pay for lack of strict schema and "completeness" of records. I definitely need to install a newer version to take advantage of it.

<a id='csv SQLite'/>

### To csv and SQLite *[export](#Data export)*

I have not re-writen another SAX content handler for csv export. I am just going to make use of dictionnary-like structure to export csv files matching the way I am going to structure the SQL database.

I will create the following files:

| File                    | Description                                          |
|:----------------------- |:---------------------------------------------------- |
| nodes.csv               | nodes attributes                                     |
| nodes_tags.csv          | nodes tags                                           |
| ways.csv                | ways attributes                                      |
| ways_nodes.csv          | references to nodes from ways                        |
| ways_tags.csv           | ways tags                                            |
| relations.csv           | relations attributes                                 |
| relations_nodes.csv     | references to nodes from relations                   |
| relations_ways.csv      | references to ways from relations                    |
| relations_relations.csv | references to relations from relations               |

Contrary to MongoDB, preparation for mass import required more work !

The following function will create all those files. It takes the dictionnary-like structure as argument:

In [57]:
import csv

def export_to_csv(dataset):
    """
    Export dataset to csv files.
    
    WARNING: the dataset is modified in-place in the operation.
    Python csv module does not support Unicode strings, we need to to encode manually in UTF-8.
    The encoding is done by function.
    
    - dataset_dict: dataset to be exported.
    """        
    def encode_utf8(dct):
        """
        Encode keys and values in UTF-8 if they are unicode strings.
        
        - dct: input dictionnary with possibly unicode strings for text keys and values.
        - return: dictionnary with text keys and values into byte strings encoded in UTF-8.
        """
        output = { }
        for key, value in dct.iteritems():
            if type(key) == type(u""):
                key = key.encode('utf-8')
            if type(value) == type(u""):
                value = value.encode('utf-8')
            output[key] = value
        return output
    
    #Process nodes
    with open('nodes.csv', 'w') as f_nodes, open('nodes_tags.csv', 'w') as f_tags:
        f_nodes_fields = ['osmid', 'longitude', 'latitude', 'userid']
        f_tags_fields = ['node_id', 'key', 'value']
        w_nodes = csv.DictWriter(f_nodes, fieldnames=f_nodes_fields)
        w_tags = csv.DictWriter(f_tags, fieldnames=f_tags_fields)
        w_nodes.writeheader()
        w_tags.writeheader()
        for node in dataset["nodes"]:
            osmid = node['osmid']
            tags = node.pop("tags")
            for tag in tags:
                tag[u'node_id'] = osmid
                w_tags.writerow(encode_utf8(tag))
            w_nodes.writerow(encode_utf8(node))
    
    #Process ways
    with open('ways.csv', 'w') as f_ways, open('ways_nodes.csv', 'w') as f_nodes, \
            open('ways_tags.csv', 'w') as f_tags:
        f_ways_fields = ['osmid', 'userid']
        f_nodes_fields = ['way_id', 'node_id']
        f_tags_fields = ['way_id', 'key', 'value']
        w_ways = csv.DictWriter(f_ways, fieldnames=f_ways_fields)
        w_nodes = csv.DictWriter(f_nodes, fieldnames=f_nodes_fields)
        w_tags = csv.DictWriter(f_tags, fieldnames=f_tags_fields)
        w_ways.writeheader()
        w_nodes.writeheader()
        w_tags.writeheader()
        for way in dataset["ways"]:
            osmid = way["osmid"]
            tags = way.pop("tags")
            nodes = way.pop("nodes")
            for tag in tags:
                tag[u'way_id'] = osmid
                w_tags.writerow(encode_utf8(tag))
            for node in nodes:
                w_nodes.writerow({'node_id': node, 'way_id': osmid})
            w_ways.writerow(encode_utf8(way))
        
    #Process relations
    with open('relations_ways.csv', 'w') as f_ways, open('relations_nodes.csv', 'w') as f_nodes,  \
            open('relations_tags.csv', 'w') as f_tags, open('relations.csv', 'w') as f_relations, \
            open('relations_relations.csv', 'w') as f_rel_rel:
        f_relations_fields = ['osmid', 'userid']
        f_tags_fields = ['relation_id', 'key', 'value']
        f_ways_fields = ['relation_id', 'way_id']
        f_nodes_fields = ['relation_id', 'node_id']
        f_rel_rel_fields = ['relation_container', 'relation_content']
        w_relations = csv.DictWriter(f_relations, fieldnames=f_relations_fields)
        w_tags = csv.DictWriter(f_tags, fieldnames=f_tags_fields)
        w_ways = csv.DictWriter(f_ways, fieldnames=f_ways_fields)
        w_nodes = csv.DictWriter(f_nodes, fieldnames=f_nodes_fields)
        w_rel_rel = csv.DictWriter(f_rel_rel, fieldnames=f_rel_rel_fields)
        w_relations.writeheader()
        w_ways.writeheader()
        w_nodes.writeheader()
        w_tags.writeheader()
        w_rel_rel.writeheader()
        for relation in dataset["relations"]:
            osmid = relation["osmid"]
            tags = relation.pop("tags")
            nodes = relation.pop("nodes")
            ways = relation.pop("ways")
            relations = relation.pop("relations")
            for tag in tags:
                tag[u'relation_id'] = osmid
                w_tags.writerow(encode_utf8(tag))
            for node in nodes:
                w_nodes.writerow({'node_id': node, 'relation_id': osmid})
            for way in ways:
                w_ways.writerow({'way_id': way, 'relation_id': osmid})
            for rel in relations:
                w_rel_rel.writerow({'relation_container': osmid, 'relation_content': rel})
            w_relations.writerow(encode_utf8(relation))

In [58]:
#Reload a dataset from JSON (the one we have is linked to MongoDB)
with open('data.json', 'r') as fobj:
    dataset_csv = json.loads(fobj.read())

#Export to csv
%time export_to_csv(dataset_csv)

#Clean dataset
del dataset_csv

CPU times: user 4.55 s, sys: 48 ms, total: 4.6 s
Wall time: 4.61 s


We now have to create the SQLite database. We need to define a schema:

In [59]:
sql_schema = """
CREATE TABLE nodes (
    osmid INTEGER PRIMARY KEY NOT NULL,
    latitude REAL,
    longitude REAL,
    userid INTEGER
);

CREATE TABLE nodes_tags (
    node_id INTEGER NOT NULL,
    key TEXT NOT NULL,
    value TEXT NOT NULL
);

CREATE TABLE ways (
    osmid INTEGER PRIMARY KEY NOT NULL,
    userid INTEGER
);

CREATE TABLE ways_tags (
    way_id INTEGER NOT NULL,
    key TEXT NOT NULL,
    value TEXT NOT NULL
);

CREATE TABLE ways_nodes (
    way_id INTEGER NOT NULL,
    node_id INTEGER NOT NULL
);

CREATE TABLE relations (
    osmid INTEGER PRIMARY KEY NOT NULL,
    userid INTEGER
);

CREATE TABLE relations_tags (
    relation_id INTEGER NOT NULL,
    key TEXT NOT NULL,
    value TEXT NOT NULL
);

CREATE TABLE relations_nodes (
    relation_id INTEGER NOT NULL,
    node_id INTEGER NOT NULL
);

CREATE TABLE relations_ways (
    relation_id INTEGER NOT NULL,
    way_id INTEGER NOT NULL
);

CREATE TABLE relations_relations (
    relation_container INTEGER NOT NULL,
    relation_content INTEGER NOT NULL
);
"""

In [60]:
#Create the database
import sqlite3
#Delete any previous version of database:
try:
    os.remove('data.sql')
except OSError:
    pass
#Close any previous connection
try:
    sql_client.close()
except NameError:
    pass
sql_client = sqlite3.connect('data.sql')
cursor = sql_client.cursor()
cursor.executescript(sql_schema)
sql_client.commit()

In [61]:
#Mass import time (inspired by https://stackoverflow.com/a/2888042/8500344)
for table, columns in [('nodes', ['osmid', 'latitude', 'longitude', 'userid']),
                       ('nodes_tags', ['node_id', 'key', 'value']),
                       ('ways', ['osmid', 'userid']),
                       ('ways_tags', ['way_id', 'key', 'value']),
                       ('ways_nodes', ['way_id', 'node_id']),
                       ('relations', ['osmid', 'userid']),
                       ('relations_tags', ['relation_id', 'key', 'value']),
                       ('relations_nodes', ['relation_id', 'node_id']),
                       ('relations_ways', ['relation_id', 'way_id']),
                       ('relations_relations', ['relation_container', 'relation_content'])]:
    with open('{}.csv'.format(table)) as fobj:
        print "Importing {}.csv [{} bytes]...".format(table, os.path.getsize("{}.csv".format(table))),
        reader = csv.DictReader(fobj)
        script = "INSERT INTO {} ({}) VALUES ({})".format(
            table, ",".join(columns), ",".join(map(lambda x: '?', columns)))
        to_db = [[row[col].decode('utf-8') for col in columns] for row in reader]
        cursor.executemany(script, to_db)
        print "done"
print "Database {} ready [{} bytes].".format('data.sql', os.path.getsize("data.sql"))

Importing nodes.csv [16052487 bytes]... done
Importing nodes_tags.csv [1381735 bytes]... done
Importing ways.csv [1230356 bytes]... done
Importing ways_tags.csv [9720022 bytes]... done
Importing ways_nodes.csv [12196915 bytes]... done
Importing relations.csv [7282 bytes]... done
Importing relations_tags.csv [75737 bytes]... done
Importing relations_nodes.csv [153563 bytes]... done
Importing relations_ways.csv [168978 bytes]... done
Importing relations_relations.csv [274 bytes]... done
Database data.sql ready [35205120 bytes].


Now the database has been populated with our dataset we can make some requests. Note that the SQLite file is around 35 Mb, smaller than the JSON file (which was 68 Mb).

Here are few requests with SQL database:

In [62]:
#Request by OpenStreetMap id:
cursor.execute("SELECT * FROM nodes WHERE osmid = ?", (8138771,))
pprint(cursor.fetchall())

[(8138771, 43.5618389, 1.3904169, 134812)]


In [63]:
#Request east-most nodes (max 3 nodes are returned)
cursor.execute("SELECT * FROM nodes WHERE longitude > 1.39 LIMIT 3")
pprint(cursor.fetchall())

[(8138771, 43.5618389, 1.3904169, 134812),
 (8138772, 43.5619922, 1.3901005, 134812),
 (53869616, 43.5702514, 1.3904678, 13384)]


In [64]:
#Refined latitude / longitude box (equivalent to City center dataset)
cursor.execute("""SELECT * FROM nodes 
               WHERE longitude > ? AND longitude < ? AND 
                     latitude > ? AND latitude < ?
               LIMIT 3""", (1.3434, 1.3496, 43.5799, 43.5838))
pprint(cursor.fetchall())

[(26691412, 43.5827846, 1.3466543, 6523296),
 (51983918, 43.582037, 1.3450115, 148173),
 (51983992, 43.582536, 1.3440651, 148173)]


In [65]:
#Find in list attributes (contrary to MongoDB we need a join here)
cursor.execute("""SELECT relations.* FROM relations
               JOIN relations_nodes ON relations_nodes.relation_id = relations.osmid
               JOIN nodes ON relations_nodes.node_id = nodes.osmid
               WHERE nodes.osmid = ?""", (265545746,))
pprint(cursor.fetchall())

[(13551, 722137)]


In [66]:
#Find in dict attributes
cursor.execute("""SELECT cities.node, nodes_tags.key, nodes_tags.value
                  FROM nodes_tags
                  JOIN                                                     --- This is a comment
                   (SELECT nodes.osmid AS node FROM nodes                  --- This is a subquery getting nodes
                    JOIN nodes_tags ON nodes_tags.node_id = nodes.osmid    --- with tags ref:INSEE and population
                    WHERE nodes_tags.key = ? OR nodes_tags.key = ?         --- we cannot use AND here so instead
                    GROUP BY nodes.osmid                                   --- we use GROUP BY, count() and HAVING
                    HAVING count(*) = 2) cities ON cities.node = nodes_tags.node_id
                  WHERE nodes_tags.key IN (?, ?, ?, ?)""", 
               (u"ref:INSEE", u"population", u"ref:INSEE", u"population", u"source:population", u"name:fr"))
pprint(cursor.fetchall())

[(26691412, u'name:fr', u'Tournefeuille'),
 (26691412, u'population', u'26674'),
 (26691412, u'ref:INSEE', u'31557'),
 (26691412, u'source:population', u'INSEE 2014'),
 (26691742, u'name:fr', u'Plaisance-du-Touch'),
 (26691742, u'population', u'17278'),
 (26691742, u'ref:INSEE', u'31424'),
 (26691742, u'source:population', u'INSEE 2014')]


In both cases (SQL and MongoDB), making requests based to tag keys and values requires relatively complex code.
Having a list of tags key / value is higly flexible because new tags may be added easilly but the cost to pay is to make queries less easy. We could have selected a subset of tags we are interested in and turn them into fields in the database.

Looking for pharmacies in Tournefeuille would require the a similar kind of request.

In [67]:
#Get major contributors - we can make arithmetics in SQL requests
cursor.execute("""SELECT userid, count(*) * 100. / (SELECT count(*) FROM nodes) as n 
                  FROM nodes GROUP BY userid ORDER BY n DESC LIMIT 10""")
pprint(cursor.fetchall())

[(1685, 68.14245767712174),
 (306172, 8.911184340943173),
 (115737, 4.42317514290312),
 (148173, 2.209955059271853),
 (1573648, 1.8659614214050333),
 (13384, 1.806491334858566),
 (125897, 1.1768080655430317),
 (571625, 0.980906603978199),
 (1626, 0.9356626949977495),
 (297482, 0.9316980225613183)]


In [68]:
#More info on user 1685
#cursor.execute("""SELECT ways_tags.key, ways_tags.value """)
cursor.execute("""SELECT ways_tags.key, ways_tags.value 
                  FROM ways_tags
                  JOIN ways ON ways.osmid = ways_tags.way_id
                  WHERE ways.userid = ?
                  GROUP BY ways.userid""", (1685,))
pprint(cursor.fetchall())

[(u'source',
  u'cadastre-dgi-fr source : Direction G\xe9n\xe9rale des Imp\xf4ts - Cadastre. Mise \xe0 jour : 2014')]


Finally, the latest MongoDB request requiring a join is simple in SQL:

In [69]:
#Now let's join
cursor.execute("""SELECT nodes.osmid FROM nodes
                  JOIN ways_nodes ON ways_nodes.node_id = nodes.osmid
                  WHERE ways_nodes.way_id = ?""", (30907996,))
pprint(cursor.fetchall())

[(343600511,),
 (3222241389,),
 (343600512,),
 (343600507,),
 (3222241387,),
 (343600513,),
 (3222241386,),
 (247420475,)]


In [70]:
#We are done with the database
sql_client.close()

We stated earlier than joining in MongoDB was more difficult (moreover it's only supported from version 3.2), but actually SQL requires more join operations due to the desing of tables.

In both cases, it is not straightforward to write requests with tags. The way OpenStreetMap uses tags is very flexible but this flexibility is also playing against consistency in an open-source database (we have found a lot of non-uniformities in the way addresses are recorded for example).

As we have kept a schema very close from the one in OpenStreetMap, (both for SQLite and MongoDB), we face the same difficulties: the way tags are recorded brings maximum flexibility but writing request becomes also much more difficult. There is then a trade-off to make between a database structure which enables maximum flexibility (and by extension a higher number of applications) or a database more strict in terms of tagging (fixed-tags for example) that would greatly improve the simplicity of use.

<a id='Conclusion'/>

## Conclusions *[top](#Top)*

Udacity instructors said thad many data analysts reports spending most of their time (up to around 75%) wrangling data, at the end of this project I can say that I now better understand that statement :)

Data wrangling is also a with a lot of trade-off:

- We can probably spend infinite amount of time auditing and cleaning large datasets. The more time is spent on wrangling and the less time will be passed on data exploration tasks. But also the more potential applications we can have with the dataset.

- The way dataset is made persistant also plays an important role. Schema may be strict and improve the ease of use during data exploration but also decreases the potential of reusability for different purposes.

In terms of techniques, there are a lot of improvements we can make:

- The dataset of the project remains very small. Even the 1 Gb dataset is small and can fit into memory. So I haven't really explored all techniques for dealing with large datasets: if I used SAX XML parsers, I also, at some point, had a Python dictionnary with the whole dataset in memory. JSON writing also requires to have a full dataset in memory. I gave a brief try to shelve and diskcache modules but I found them very slow, probably a misuse.

- String comparison may fail, due to different case, accented characters or not, use of - in names,... We could probably increase the flexibility of cleaning (and auting) steps by using some fuzzy comparisons. In addition,  caution is needed to properly deal with unicode strings, encoding and decoding ([Ned Batchelder talk at Pycon 2012 is worth the detour.](https://www.youtube.com/watch?v=sgHbC6udIqc))

- Finally, my first experience in SQL helps but is not enough when it comes to bigger requests (with subrequests, pivoting...) so I need more practice. MongoDB is the first no-SQL database I use, so I need to practice even more with aggregation pipelines and lookups once I have updated my configuration. I have also seen that it exists a lot of Object Relational Mapping systems based on Python and Mongodb. This would probably worth a try when I have some spare time :)

<a id="Appendix"/>

## Appendix *[top](#Top)*

### References

[OpenStreetData wiki](http://wiki.openstreetmap.org/wiki/Main_Page)<hr>
[INSEE](https://www.insee.fr/en/accueil) is French National Institute of Statistics and Economic Information. In this project, it is used as *gold* standard.<hr>
[Pages Jaunes](https://www.pagesjaunes.fr/annuaire/tournefeuille-31/pharmacies) used as another *gold* standard.<hr>
Validating XML tree with [XML Schema](https://www.w3schools.com/xml/schema_intro.asp) can be done with [lxml](http://lxml.de/validation.html) library. This technique has not been used here as the structure of XML is simple enough. Additionaly, XML Schema validation requires to have XML data into memory and may not be suitable for large files like the ones we might have here.<hr>
Get line number in a content handler with SAX parser on [StackOverflow](https://stackoverflow.com/a/15477803/8500344).<hr>
Display lists as html tables in notebook on [StackOverflow](https://stackoverflow.com/a/42323522/8500344).<hr>
Fuzzy string matching blog post on [streamhacker.com](https://streamhacker.com/2011/10/31/fuzzy-string-matching-python/).<hr>
MongoDB from Python: [pymongo](http://api.mongodb.com/python/current/tutorial.html) tutorial.<hr>
MongoDB [manual](https://docs.mongodb.com/manual/).<hr>
Geometry algorithms for [inclusion](http://geomalgorithms.com/a03-_inclusion.html) checks.<hr>
StackOverflow for [MongoDB](https://stackoverflow.com/questions/tagged/mongodb).<hr>
Ned Batchelder talk at Pycon 2012 on [YouTube](https://www.youtube.com/watch?v=sgHbC6udIqc): How to stop unicode pain ?<hr>
Python SQLite import from csv on [StackOverflow](https://stackoverflow.com/a/2888042/8500344).<hr>